## Refeerences

Adverserial probabilities from https://www.kaggle.com/joatom/a-test-like-validation-set
Other References are inline.

In [1]:
#!conda install -c conda-forge pandas-profiling -y
#!pip install eli5

In [2]:
%matplotlib inline

In [3]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold, StratifiedKFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import roc_curve, auc, confusion_matrix, accuracy_score, precision_recall_curve
import eli5
from eli5.sklearn import PermutationImportance

import warnings
warnings.filterwarnings('ignore')

from pandas_profiling import ProfileReport

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [4]:
revs2=pd.read_csv('/kaggle/input/trends-assessment-prediction/reveal_ID_site2.csv')
data = pd.read_csv('/kaggle/input/trends-assessment-prediction/loading.csv')
y_data = pd.read_csv('/kaggle/input/trends-assessment-prediction/train_scores.csv')

In [5]:
s2data = data.merge(revs2)
s1data = data.merge(y_data['Id'])

y_data['is_test'] = 1
train_test = data.merge(y_data[['Id','is_test']], how ='left').fillna(0)
#train_test_ids=train_test.pop('Id')

In [6]:
#s2data.describe()

In [7]:
#s1data.describe()

# Helpers

In [8]:
# setup KFold
splits = 3
#repeats = 2
rskf = StratifiedKFold(n_splits=splits, random_state=2020, shuffle=True)

In [9]:
# https://www.kaggle.com/ynouri/random-forest-k-fold-cross-validation
def compute_roc_auc(X, y, index, clf):
    y_predict = clf.predict_proba(X.iloc[index])[:,1]
    print(y_predict)
    fpr, tpr, thresholds = roc_curve(y.iloc[index], y_predict)
    auc_score_roc = auc(fpr, tpr)
    # http://www.davidsbatista.net/blog/2018/08/19/NLP_Metrics/
    precision, recall, thresholds = precision_recall_curve(y.iloc[index], y_predict)
    auc_score_prc = auc(recall, precision)
    
    return y_predict, auc_score_roc, auc_score_prc

In [10]:
def preds(all_data, av_y, oof_df = None):
    features=list(set(all_data.columns)-set(['Id']))
    # http://fastml.com/adversarial-validation-part-one/
    clf = RandomForestClassifier(n_estimators = 100, n_jobs = -1, random_state=2020)

    fprs, tprs, scores_roc_train, scores_roc_valid, scores_prc_train, scores_prc_valid = [], [], [], [], [], []
    # https://github.com/zygmuntz/adversarial-validation/blob/master/numerai/sort_train.py
    predictions = np.zeros(av_y.shape[0])
    
    if isinstance(oof_df,pd.DataFrame):
        oof_df['av_prob']=0
        
    for (i_train, i_valid), i in zip(rskf.split(all_data[features],av_y),range(splits)):
        print('Split', i)
        clf.fit(all_data[features].iloc[i_train], av_y.iloc[i_train])

        # score
        _, auc_score_roc_train, auc_score_prc_train = compute_roc_auc(all_data[features], av_y, i_train, clf)
        y_predict, auc_score_roc, auc_score_prc = compute_roc_auc(all_data[features], av_y, i_valid, clf)
        predictions[i_valid] = y_predict

        #oof
        if isinstance(oof_df,pd.DataFrame):
            oof_df['av_prob'] += clf.predict_proba(oof_df.iloc[oof_df.index][features])[:,1]/splits
        
        scores_roc_train.append(auc_score_roc_train)
        scores_roc_valid.append(auc_score_roc)
        scores_prc_train.append(auc_score_prc_train)
        scores_prc_valid.append(auc_score_prc)

        # Feature Importance
        ## https://towardsdatascience.com/running-random-forests-inspect-the-feature-importances-with-this-code-2b00dd72b92e
        clf.score(all_data[features].iloc[i_valid], av_y.iloc[i_valid])
        rf_feature_importances = pd.DataFrame(clf.feature_importances_,
                                           index = features,
                                           columns=['importance']).sort_values('importance', ascending=False)
        display(rf_feature_importances.head(10))

        # Permutation Importance
        #permImp = PermutationImportance(clf, random_state=2021).fit(all_data.iloc[i_valid], av_y.iloc[i_valid]) 
        #display(eli5.show_weights(permImp, feature_names = all_data.columns.tolist()))

    print('Mean Accuracy roc:', np.mean(scores_roc_valid))
    print('Mean Accuracy precision recal:', np.mean(scores_roc_valid))
    
    return predictions, oof_df


# AV for train test

In [11]:

av_y_tt = train_test.pop('is_test')

In [12]:

# log regression
scores = cross_val_score(LogisticRegression(random_state=2020, solver='lbfgs',max_iter=1000), train_test, av_y_tt, cv=rskf, scoring='roc_auc') #'f1'
print("Log Regression Accuracy (RoC): %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), 'AV LogReg'))

# random forrest
scores = cross_val_score(RandomForestClassifier(n_estimators = 100, n_jobs = -1, random_state=2020), train_test, av_y_tt, cv=rskf, scoring='roc_auc') #'f1'
print("Random Forrest Accuracy (RoC): %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), 'AV RandomForestClas'))

Log Regression Accuracy (RoC): 0.51 (+/- 0.01) [AV LogReg]
Random Forrest Accuracy (RoC): 0.71 (+/- 0.00) [AV RandomForestClas]


In [13]:
train_test['is_test_prob'], _ = preds(train_test, av_y_tt)

Split 0
[0.13 0.94 0.88 ... 0.29 0.93 0.13]
[0.52 0.48 0.51 ... 0.57 0.39 0.58]


,importance
IC_20,0.121890
IC_21,0.040354
IC_18,0.038931
IC_11,0.038508
IC_28,0.038123
IC_05,0.037950
IC_30,0.037681
IC_16,0.037402
IC_24,0.036447
IC_08,0.035795


Split 1
[0.78 0.82 0.14 ... 0.14 0.92 0.88]
[0.49 0.71 0.27 ... 0.73 0.79 0.42]


,importance
IC_20,0.124465
IC_21,0.042203
IC_18,0.040982
IC_28,0.038431
IC_11,0.038029
IC_30,0.036868
IC_05,0.036611
IC_13,0.035894
IC_02,0.035570
IC_17,0.035450


Split 2
[0.82 0.75 0.19 ... 0.35 0.11 0.76]
[0.26 0.76 0.76 ... 0.51 0.3  0.75]


,importance
IC_20,0.129842
IC_21,0.041943
IC_18,0.038532
IC_11,0.037547
IC_28,0.037188
IC_05,0.036355
IC_07,0.036156
IC_24,0.036043
IC_30,0.035853
IC_16,0.035710


Mean Accuracy roc: 0.711460297681508
Mean Accuracy precision recal: 0.711460297681508


In [14]:
train_test['is_test']=av_y_tt
train_test.groupby(['is_test']).describe()[['is_test_prob']]

is_test_prob                                                  
               count      mean       std   min   25%   50%   75%   max
is_test                                                               
0.0           5877.0  0.445020  0.146392  0.08  0.34  0.44  0.55  0.86
1.0           5877.0  0.555603  0.131807  0.10  0.47  0.57  0.65  0.94

In [15]:
train_test = train_test.drop('is_test',axis=1)

# AV for site2

In [16]:
s2data['is_site2']=1
s1data['is_site2']=0

train_s2test = s2data.append(s1data)
#train_s2test_ids = train_s2test.pop('Id')

av_y = train_s2test.pop('is_site2')

In [17]:

# log regression
scores = cross_val_score(LogisticRegression(random_state=2020, solver='lbfgs',max_iter=1000), train_s2test, av_y, cv=rskf, scoring='roc_auc') #'f1'
print("Log Regression Accuracy (RoC): %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), 'AV LogReg'))

# random forrest
scores = cross_val_score(RandomForestClassifier(n_estimators = 100, n_jobs = -1, random_state=2020), train_s2test, av_y, cv=rskf, scoring='roc_auc') #'f1'
print("Random Forrest Accuracy (RoC): %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), 'AV RandomForestClas'))

Log Regression Accuracy (RoC): 0.48 (+/- 0.01) [AV LogReg]
Random Forrest Accuracy (RoC): 0.81 (+/- 0.01) [AV RandomForestClas]


In [18]:
train_s2test['p'], train_test = preds(train_s2test, av_y, train_test)

Split 0
[0.69 0.78 0.67 ... 0.01 0.01 0.  ]
[0.24 0.32 0.11 ... 0.08 0.08 0.08]


,importance
IC_20,0.142045
IC_18,0.059396
IC_11,0.046002
IC_10,0.039022
IC_05,0.038390
IC_09,0.037573
IC_04,0.035975
IC_16,0.035623
IC_06,0.035223
IC_30,0.035077


Split 1
[0.64 0.68 0.7  ... 0.03 0.02 0.  ]
[0.21 0.44 0.16 ... 0.19 0.02 0.02]


,importance
IC_20,0.129514
IC_18,0.055355
IC_11,0.047623
IC_21,0.040120
IC_16,0.039416
IC_05,0.037912
IC_28,0.037778
IC_04,0.036049
IC_30,0.035712
IC_24,0.034507


Split 2
[0.68 0.73 0.62 ... 0.03 0.   0.01]
[0.04 0.2  0.07 ... 0.07 0.02 0.01]


,importance
IC_20,0.128818
IC_18,0.061521
IC_05,0.043062
IC_11,0.041748
IC_24,0.038201
IC_16,0.038098
IC_08,0.037080
IC_02,0.037076
IC_07,0.036934
IC_06,0.035949


Mean Accuracy roc: 0.8193265871943468
Mean Accuracy precision recal: 0.8193265871943468


In [19]:
train_s2test['is_s2']=av_y
train_s2test.groupby(['is_s2']).describe()[['p']]

p                                                 
        count      mean       std  min   25%   50%   75%   max
is_s2                                                         
0      5877.0  0.076338  0.079247  0.0  0.02  0.05  0.10  0.57
1       510.0  0.216588  0.143897  0.0  0.09  0.19  0.32  0.67

In [20]:
train_test.columns.av_prob  = 'site2_prob'

In [21]:
# override with validation data to not overfit to known train_tests2 data

In [22]:
train_test=train_test.merge(train_s2test[['Id', 'p']], how='left')

In [23]:
train_test['is_site2_prob'] = train_test['p'].combine_first(train_test['av_prob'])

In [24]:
train_test[['Id','is_test_prob','is_site2_prob']].to_csv('test_s2_probs.csv', index=False)